[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DoranLyong/Awesome-Tensor-Architecture/blob/main/pytorch_reference/simple_reference/06_PyTorch_Acceleration_and_Optimization/04_Quantization.ipynb)



# Quantization (p.178) - ([ref](https://pytorch.org/docs/stable/quantization.html))
```NN```s are implemented as ```computational graphs```, and their computations often use ```32-bit``` (or 64-bit) floating-point numbers. <br/>
Compute to ```lower-precision``` numbers and still acheive ```comparable results``` by applying quantization.


__Quantization__ refers to techniques for ```computing``` and ```accessing memory``` with ```lower-precision``` data.

(effectiveness)
* decreasing ```model size```
* reducing ```memory bandwidth```
* faster inference due to savings in memory bandwidth
* faster computing with ```int8``` arithmetic

In [4]:
import torch 
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F 

We use CPU backend for this example.

### Basic Model (p.179)

In [5]:
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d( F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d( F.relu(self.conv2(x)), 2)
        x = x.view(-1, int(x.nelement() / x.shape[0]))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
x = torch.randn(4, 3, 32, 32)

model = LeNet5()
scores = model(x)

(Above) all ```computations``` and ```memory``` are implemented as ```float32``` in basic.

In [8]:
# inspect the data types of our model's parameteres 

for n, p in model.named_parameters():

    print(n, ": ", p.dtype)

conv1.weight :  torch.float32
conv1.bias :  torch.float32
conv2.weight :  torch.float32
conv2.bias :  torch.float32
fc1.weight :  torch.float32
fc1.bias :  torch.float32
fc2.weight :  torch.float32
fc2.bias :  torch.float32
fc3.weight :  torch.float32
fc3.bias :  torch.float32


### Quick Quantization Method (p.180)
* reduce the model to half precision using ```half()``` method.

In [11]:
model = model.half()

for n, p in model.named_parameters():
    print(n, ": ", p.dtype)

conv1.weight :  torch.float16
conv1.bias :  torch.float16
conv2.weight :  torch.float16
conv2.bias :  torch.float16
fc1.weight :  torch.float16
fc1.bias :  torch.float16
fc2.weight :  torch.float16
fc2.bias :  torch.float16
fc3.weight :  torch.float16
fc3.bias :  torch.float16


Now our ```computation``` and ```memory values``` are ```float16```.

***
we ```don't``` want to ```quantize``` every computation ```in the same way```. <br/>
and we may need to quantize beyond ```float16``` values. 

## Three other modes of quantization (p. 181)
1. dynamic quantization 
2. post-training static quantization
3. quantization-aware training (QAT)

### Dynamic Quantization:
when ```throughput``` is limited by compute or memory bandwidth for weights. <br/> 

This is often true for __LSTM__, __RNN__, __Bidirectional Encoder Representations from Transformers__ (BERT), or __Transformer__ networks.

### Post-training Static Quantization:
when ```throughput``` is limited by ```memory bandwidth``` for ```activations```. <br/>
Often applies for ```CNN```s.

### QAT 
when ```accuracy requirements``` cannot be achieved by static quantization.

All types above convert ```weights``` into ```int8```.<br/>
They vary in handle activations and memory access.

***

### (ex1) Dynamic quantization (p.181)
- it is the easiest type
- it ```converts``` the ```activations``` to ```int8``` on the fly.

Computations use efficient ```int8``` values, <br/> 
but the ```activations``` are ```read and written``` to memory in ```floating-point``` format.

In [12]:
import torch.quantization

quantized_model = torch.quantization.quantize_dynamic(  model,              # pass our model. 
                                                        {torch.nn.Linear},  # specify the quantized layers.
                                                        dtype=torch.qint8   # specify the quantization level.
                                                    )

#### ★★★ (NOTICE) 
Quantization depends on the backend being used to run quantized models. <br/>
Currently, quantized operators are supported for CPU inference only in the following backends: 
* x86 (fbgemm)
* ARM (qnnpack)

However, __QAT__ occurs in full floating point and can run on either GPUs or CPUs.

### (ex2) Post-training Static Quantization (p.182)
* to ```further reduce``` latency by observing the distributions of different activations ```during training```
* and by deciding ```how those activations should be``` quantized at the time of inference.

This type of quantization allows us to pass quantized values between operations without converting back and forth between ```float```s and ```int```s in memory.

In [13]:
static_quant_model = LeNet5()

static_quant_model.qconfig = torch.quantization.get_default_qconfig('fbgemm') # configure the backend to use x86 (fbgemm)

torch.quantization.prepare( static_quant_model, inplace=True)  # to insert observers to calibrate the model and collect statistics.
torch.quantization.convert( static_quant_model, inplace=True)  # to convert the model to a quantized version. 

/home/milky/anaconda3/envs/py38/lib/python3.8/site-packages/torch/ao/quantization/observer.py:172: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/home/milky/anaconda3/envs/py38/lib/python3.8/site-packages/torch/ao/quantization/observer.py:1107: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


LeNet5(
  (conv1): QuantizedConv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), scale=1.0, zero_point=0)
  (conv2): QuantizedConv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), scale=1.0, zero_point=0)
  (fc1): QuantizedLinear(in_features=400, out_features=120, scale=1.0, zero_point=0, qscheme=torch.per_channel_affine)
  (fc2): QuantizedLinear(in_features=120, out_features=84, scale=1.0, zero_point=0, qscheme=torch.per_channel_affine)
  (fc3): QuantizedLinear(in_features=84, out_features=10, scale=1.0, zero_point=0, qscheme=torch.per_channel_affine)
)

### (ex3) QAT
* typically results in the best accuracy 

All ```weights``` and ```activations``` are "```fake quantized```" during the forward and backward pass of ```training```. <br/>
```Float values``` are ```rounded``` to the ```int8``` equivalent, but the computations are ```still``` done in ```floating point```. 

(i.e) the ```weight adjustments``` are made "```aware```" that they will be quantized during training.

In [14]:
qat_model = LeNet5()
qat_model.qconfig = torch.quantization.get_default_qat_qconfig('fbgemm')

torch.quantization.prepare_qat( qat_model, inplace=True)
torch.quantization.convert( qat_model, inplace=True)

/home/milky/anaconda3/envs/py38/lib/python3.8/site-packages/torch/ao/quantization/observer.py:172: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/home/milky/anaconda3/envs/py38/lib/python3.8/site-packages/torch/ao/quantization/utils.py:156: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


LeNet5(
  (conv1): QuantizedConv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), scale=1.0, zero_point=0)
  (conv2): QuantizedConv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), scale=1.0, zero_point=0)
  (fc1): QuantizedLinear(in_features=400, out_features=120, scale=1.0, zero_point=0, qscheme=torch.per_channel_affine)
  (fc2): QuantizedLinear(in_features=120, out_features=84, scale=1.0, zero_point=0, qscheme=torch.per_channel_affine)
  (fc3): QuantizedLinear(in_features=84, out_features=10, scale=1.0, zero_point=0, qscheme=torch.per_channel_affine)
)